In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from urllib.parse import urljoin, urlparse
import mysql.connector
import logging
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

In [2]:
def get_all_page2():
    urls=[]
    page_number = 1
    
    for i in range(50):
        i = f"https://books.toscrape.com/catalogue/page-{page_number}.html"
        page_number += 1
        urls.append(i)
    return urls

In [3]:
# Fonction pour extraire les détails d'un livre
def extract_book_details2(book):
    
    # book c'est la section d'une page qui contient un livre. c'est dans cette section on va extraire le lien pour 
    # entrer dans la page d'un livre
    
    #book_url c'est le lien pour entrer sur la page d'un livre 
    book_url = book.find('div', class_='image_container').a["href"]
    
    # URL de base pour arriver sur un livre en particulier
    base_url = 'https://books.toscrape.com/catalogue/'
    
    #former le lien complet du livre sur la page en concatenant book_url et base_url
    lien_complet = urljoin(base_url,  book_url)
    url_book = lien_complet
    #print(url_book)
    
    # extraire les informations du livre dans sa page
    response = requests.get(lien_complet) #ouvrir la page 
    soup = BeautifulSoup(response.text, 'html.parser')  # transformer en beautifulsoup

    title = soup.find('h1').text.strip() # le titre du livre
    p = soup.find('p', class_='price_color').text.strip() # le prix tout taxe
    priceTTC = re.search(r'\d+\.\d+', p).group() # retrait du symbole de l'euro
    
    #extraire la disponibilite du produit
    chaine = soup.find('p', class_='instock availability').text.strip()
    availability = re.search(r'^\w+\s\w+', chaine).group()
    
    #extraire la quantite du produit
    stock = re.search(r'\d+', chaine).group()
    
    #extraire le raiting
    rating_class = soup.find('p', class_='star-rating')['class'][1]
    rating = rating_class.strip() 
    
    #extraire le lien vers l'image du livre
    base_url2 = 'https://books.toscrape.com/media/'  # url de base pour les images
    im = soup.find('div', class_='item active').img["src"]
    chemin_relatif2 = '/'.join(im.split('/')[3:]) # Extraire le chemin relatif de l'image
    image_url = urljoin(base_url2, chemin_relatif2)
    
    #extraire la categorie du produit
    li_tags = soup.find('ul', class_='breadcrumb')
    li = li_tags.find_all('li')
    # L'avant-dernier li
    category = li[-2].get_text(strip=True)
    #print(category)
    
    
    #extraire la taxe
    t = soup.find('table', class_='table table-striped')
    tr=t.find_all('tr')
    taxe = tr[4].td.get_text(strip=True)
    #print(taxe)
    
    # extraire number of reviews
    n= soup.find('table', class_='table table-striped')
    tr=n.find_all('tr')
    reviews = tr[6].td.get_text(strip=True)
    #print(reviews)
    
    # Créer une liste pour stocker les données de chaque élément
    data = []
    data.append([title, priceTTC, availability,stock, image_url,url_book, rating, category])
    with open('produits_extrait.csv', mode='a', newline='',encoding='utf-8') as file:
        writer = csv.writer(file)
        #writer.writerow(['title', 'price', 'availability', 'image_url','book_url', 'Rating','category'])
        writer.writerows(data)

    return data

In [ ]:
def parse_all_page2():
    page = get_all_page2()# recuperer la liste des liens vers toutes les pages
    n=[]
    for p in page: # p c'est le lien d'une page
        r = requests.get(p) # ouvrir une page
        soup = BeautifulSoup(r.text, 'html.parser')
        books = soup.find_all('article', class_='product_pod') # recupere les sections de tous les livres de la page
        
        print(f"on scrape{p}\n")
        for b in books: # pour un livre particulier b
            n.append(extract_book_details2(book=b)) # extraire les details de ce livre et ajouter a la liste n
        
    print(len(n))
    return n
datasetfin = []
datasetfin=parse_all_page2()

on scrapehttps://books.toscrape.com/catalogue/page-1.html

on scrapehttps://books.toscrape.com/catalogue/page-2.html



In [ ]:
def get_categorie():
    # URL de base
    base_url = 'https://books.toscrape.com/'

    # Récupérer la page d'accueil
    response = requests.get(base_url + 'index.html')
    soup = BeautifulSoup(response.text, 'html.parser')

    # Récupérer la liste des catégories et supprimer la première catégorie
    categories = soup.find('ul', class_='nav-list').find_all('li')
    categories.pop(0)  # Supprimer la première catégorie
    nomsCategories = []
    for cat in categories:
        nomsCategories.append(cat.a.text.strip())
    return nomsCategories
c = get_categorie()
# print(c)

In [ ]:
data_client = pd.read_csv('produits_extrait.csv',names=['title', 'price', 'availability','stock', 'image_url','book_url' ,'rating', 'category'],header=None,index_col=False)

data_client['rating'] = data_client['rating'].replace({'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5})
data_client.head(50)
#print(data_client['category'])

In [ ]:
# verifier les donnees manquantes avec `isna()`
missing_data = data_client.isna().sum()

# afficher les donnees manquantes
print("Données manquantes par colonne :")
print(missing_data)

# chercher les lignes avec les donnees manquantes `any()`
rows_with_missing_data = data_client.isnull().any(axis=1)

# afficher le nombre de ligne avec les donnees manquantes
print("\nNombre de lignes avec des données manquantes :", rows_with_missing_data.sum())


In [ ]:
#les 5 premiers
p = data_client.sort_values(by='price').head(10)
p

# Créer un graphique à barres
plt.figure(figsize=(15, 8))
bars = plt.bar(p['title'], p['price'], width=0.5,color=['lightblue', 'lightgreen', 'lightcoral', 'lightsalmon', 'lightseagreen'])

# Annoter chaque barre avec le montant de la vente
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height}', ha='center', va='bottom')

# Abbreviation de l'axe des x
abbreviated_titles = [title[:20] + '...' if len(title) > 20 else title for title in p['title']]
plt.xticks(range(len(p)), abbreviated_titles, rotation=45, ha='right')

plt.xticks(range(len(p)), rotation=45, ha='right') 
plt.xlabel('Livres', fontsize=12, weight='bold')
plt.ylabel('Prix', fontsize=12, weight='bold')

# Ajouter une grille pour une meilleure lisibilité
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


In [ ]:

#trier le dataframe par ordre decroissant
p = data_client.sort_values(by='price', ascending=False).head(10)
p
# Créer un graphique à barres
plt.figure(figsize=(15, 8))
bars = plt.bar(p['title'], p['price'], width=0.5, color = ['lightgreen', 'lightblue', 'gray', 'brown', 'lightgray'])

# Annoter chaque barre avec le montant de la vente
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height}', ha='center', va='bottom')

# Abbreviation de l'axe des x
abbreviated_titles = [title[:20] + '...' if len(title) > 20 else title for title in p['title']]
plt.xticks(range(len(p)), abbreviated_titles, rotation=45, ha='right')

# Ajouter une grille pour une meilleure lisibilité
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.xticks(range(len(p)), rotation=45, ha='right') 
plt.xlabel('Livres', fontsize=12, weight='bold')
plt.ylabel('Prix', fontsize=12, weight='bold')
plt.show()

In [ ]:

#trier le dataframe par ordre decroissant
p = data_client.sort_values(by='rating', ascending=False).head(10)
p
# Créer un graphique à barres
plt.figure(figsize=(15, 8))
bars = plt.bar(p['title'], p['rating'], width=0.5,color=['green', 'black', 'red', 'blue', 'yellow'])

# Annoter chaque barre avec le montant de la vente
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height}', ha='center', va='bottom')
    
# Abbreviation de l'axe des x
abbreviated_titles = [title[:20] + '...' if len(title) > 20 else title for title in p['title']]
plt.xticks(range(len(p)), abbreviated_titles, rotation=45, ha='right')

# Ajouter une grille pour une meilleure lisibilité
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.xticks(range(len(p)), rotation=45, ha='right') 
plt.xlabel('Livres', fontsize=12, weight='bold')
plt.ylabel('Rating', fontsize=12, weight='bold')
plt.savefig('les_10_produits_les_plus_recommendes.png')
plt.show()

In [ ]:
#les 5 premiers
p = data_client.sort_values(by='rating').head(10)
p
# Créer un graphique à barres
plt.figure(figsize=(15, 8))
bars = plt.bar(p['title'], p['rating'], width=0.5,color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])

# Annoter chaque barre avec le montant de la vente
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height}', ha='center', va='bottom')

# Abbreviation de l'axe des x
abbreviated_titles = [title[:20] + '...' if len(title) > 20 else title for title in p['title']]
plt.xticks(range(len(p)), abbreviated_titles, rotation=45, ha='right')

# Ajouter une grille pour une meilleure lisibilité
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.xticks(range(len(p)), rotation=45, ha='right') 
plt.xlabel('Livres', fontsize=12, weight='bold')
plt.ylabel('Rating', fontsize=12, weight='bold')
plt.show()

In [ ]:
# Afficher un diagramme circulaire des catégories ayant le plus grand nombre de livres en stock
plt.figure(figsize=(10, 10))
plt.pie(p, labels=p.index, autopct="%1.1f%%", labeldistance=1.1, textprops={'fontsize': 12, 'weight': 'bold'})
plt.show()

In [ ]:
# Afficher un diagramme circulaire des catégories ayant le plus grand nombre de livres en stock
plt.figure(figsize=(8, 8))

# Définir les couleurs à utiliser dans le graphique
colors = plt.cm.tab10.colors

# Diagramme circulaire avec légende et explosion des tranches
patches, texts, autotexts = plt.pie(p, labels=p.index, autopct="%1.1f%%", labeldistance=1.1, colors=colors, textprops={'fontsize': 12, 'weight': 'bold'})

# Ajouter une légende expliquant les couleurs
plt.legend(patches, p.index, loc="center left", bbox_to_anchor=(1, 0.5))

# Afficher le graphique
plt.show()

In [ ]:
p=data_client.groupby("category")["price"].mean().sort_values(ascending = False).head(3)
p

plt.figure(figsize = (10,10))
plt.pie(p,labels =p.index, rotatelabels =True,autopct="%1.1f%%" , textprops={'fontsize': 12, 'weight': 'bold'});
# Ajouter une légende expliquant les couleurs
plt.legend(patches, p.index, loc="center left", bbox_to_anchor=(1, 0.5))
plt.savefig('Les_3_catégories_ayant_les_produits_les_plus_coûteux.png')